# Processing the final dataset



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"

Please restart Jupyter kernel after installation if you encounter any errors when importing geemap.


ModuleNotFoundError: No module named 'pkg_resources'

In [3]:
# region of interest is Amazonia
# roi = ee.FeatureCollection(f"{data_folder}/biomes_br").first().geometry()
# For specific biomes, the feature "CD_Bioma" is:
# 0 = Amazonia
# 1 = Caatinga
# 2 = Cerrado
# 3 = Mata Atlantica
# 4 = Pampa
# 5 = Pantanal

In [3]:
age_agbd = ee.Image(f"{data_folder}/age_agbd")
categorical = ee.Image(f"{data_folder}/categorical")
sur_cover = ee.Image(f"{data_folder}/sur_cover")
one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask")
land_use = ee.Image(f"{data_folder}/land_use")
land_use_5_years = ee.Image(f"{data_folder}/land_use_5_years")
land_use_10_years = ee.Image(f"{data_folder}/land_use_10_years")
land_use_15_years = ee.Image(f"{data_folder}/land_use_15_years")

zuquim_soil = ee.Image(f"{data_folder}/raw/zuquim_amazon_soil_SB")
cwd = ee.Image(f"{data_folder}/raw/cwd_chave").float()
yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
yearly_SI = ee.Image(f"{data_folder}/yearly_SI")


# unified_data = age_agbd.addBands([land_use_10_years, cwd]).updateMask(land_use_10_years.select('last_LU')).updateMask(one_hectare_mask)
unified_data = (
    age_agbd.addBands(
        [land_use_5_years, cwd, categorical, sur_cover, yearly_mean_prec, yearly_SI]
    )
    .updateMask(land_use_5_years.select("last_LU"))
    .updateMask(one_hectare_mask)
)

NameError: name 'data_folder' is not defined

In [2]:
# ## stratify sample points per feature
# stratified = final.stratifiedSample(
#     numPoints = 10000,
#     classBand = 'ecoreg',
#     geometries = True
# )

unified_data_fc = unified_data.sample(
    geometry = unified_data.geometry(), projection = age_agbd.projection(), scale = 30, geometries = True
)

# Export an ee.FeatureCollection as an Earth Engine asset.
task = ee.batch.Export.table.toDrive(
    collection=unified_data_fc, description="unified_data"
)
task.start()

NameError: name 'unified_data' is not defined

In [31]:
img = unified_data
name = "unified_data"

# Create the export task
task = ee.batch.Export.image.toDrive(
    image=img,
    description=f"{name}",
    crs="EPSG:4326",
    maxPixels=4e12,
    region=img.geometry(),
)

# Start the export task
task.start()

In [24]:
# # Select the WORLDCLIM dataset and specifically the 'bio01' band which represents annual mean temperature.
# dataset = ee.Image('WORLDCLIM/V1/BIO')
# annual_mean_temperature = dataset.select('bio15').multiply(0.1)
# https://developers.google.com/earth-engine/datasets/catalog/WORLDCLIM_V1_BIO#bands
# # # Define visualization parameters.
# # vis_params = {
# #     'min': 0,
# #     'max': 15,
# #     'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
# # }


# # Map = geemap.Map()

# # # Set the map center and zoom level.
# # Map.centerObject(age_agbd)

# # # Add the layer to the map.
# # Map.addLayer(annual_mean_temperature, vis_params, 'Annual Mean Temperature')
# # Map